In [15]:
# Jupyter notebook needs this or else it will crash
from datetime import datetime
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [16]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [17]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:

        try:

            if connection:
                # Add an edge labelled 'connection' with a start time of <time>
                g.V().has('name', name1).as_("a").not_( # NEGATE 
                    __.bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
                        __.has('start', P.lte(time)),
                        __.or_(
                            __.hasNot('end'),
                            __.has('end', P.gt(time))
                        )
                    )
                ).V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

            else:
                # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
                g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()
        
            success = True

        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to set {connection} connection between {name1} and {name2} at {time}", e)

In [18]:
def add_component(name: str) -> None:
    """
    Add a vertex to the graph with a 'name' property of <name>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.addV().property('name', name).iterate()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to add component of name {name}", e)


In [19]:
def add_type(type: str) -> None:
    """
    Add a vertex to the graph with a 'type' property of <type>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.addV().property('name', type).iterate()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to add type vertex of name {type}", e)


def set_type(name: str, type: str) -> None:
    """
    Connect the vertex labelled with <name> to a vertex labelled <type> with an edge going into the <type> vertex labelled with "type".

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:
        try:
            g.V().has('name', name).as_("a").V().has('name', type).as_("b").addE("type").from_("a").to("b").iterate()
            success = True
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Failed to set type of {name} to {type}", e)

In [20]:
# This will put it in V-E-V-E-V-...-V form as a list per path.
def find_paths(name1: str, name2: str, avoid_type: str, time: float):
    """
    Given two vertices labelled with <name1> and <name2>, return the paths that connect the vertices by edges that were active at <time> as a list.

    Avoid vertices of type avoid_type.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """
    while True:
        try:
            return g.V().has('name', name1).repeat(
            __.bothE('connection').and_(
                __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
                __.or_(
                    __.hasNot('end'),           # end time doesn't have to exist 
                    __.has('end', P.gt(time))  # OR end time must be greater than <time>
                )
            ).otherV().not_(__.outE('type').inV().has('name', avoid_type)).simplePath()
        ).until(__.has('name', name2)).path().toList()
        except GremlinServerError as e:
            print(f"{datetime.now().strftime('%H:%M:%S')} ERROR: Could not find paths between {name1} and {name2} at time {time} avoiding {avoid_type}.", e)
    

In [21]:
# number of HIRAX dishes
dishes = 1024

# Connections at every 3.
max_time = 30
connections = [(i, not bool(i % 2)) for i in range(0, max_time // 3 + 1, 3)]

# Correlator node name
cor = 'COR000000'

In [22]:
# Clear vertices
# Doing this does NOT use indexing.
g.V().drop().iterate()
g.E().drop().iterate()

"""
(Temporary) Naming scheme:
 - COR######: Correlator input
 - ANT######: Antenna
 - DPF######: Dual-Polarization Feed
 - BLN######: (Active) Balun
 - RFT######: RFoF transmitter
 - OPF######: Optical Fiber
 - RFR######: RFoF receiver
 - ADC######: Analog-to-Digital converter
"""

# Set up the types
types = ['COR', 'ANT', 'DPF', 'BLN', 'RFT', 'OPF', 'RFR', 'ADC']
for type in types:
    add_type(type)


# Add a correlator input node
add_component(cor)
set_type(cor, 'COR')

# Add the components and connect them at different times.
for i in range(1, dishes + 1):



    # The names of the components to refer to
    ant = f'ANT{str(i).zfill(6)}'
    dpf = f'DPF{str(i).zfill(6)}'
    bln = (f'BLN{str(2 * i - 1).zfill(6)}', f'BLN{str(2 * i).zfill(6)}')
    rft = (f'RFT{str(2 * i - 1).zfill(6)}', f'RFT{str(2 * i).zfill(6)}')
    opf = (f'OPF{str(2 * i - 1).zfill(6)}', f'OPF{str(2 * i).zfill(6)}')
    rfr = (f'RFR{str(2 * i - 1).zfill(6)}', f'RFR{str(2 * i).zfill(6)}')
    adc = (f'ADC{str(2 * i - 1).zfill(6)}', f'ADC{str(2 * i).zfill(6)}')

    now = datetime.now()

    add_component(ant)
    add_component(dpf)

    set_type(ant, 'ANT')
    set_type(dpf, 'DPF')

    for ind in (0, 1):
        add_component(bln[ind])
        add_component(rft[ind])
        add_component(opf[ind])
        add_component(rfr[ind])
        add_component(adc[ind])

        set_type(bln[ind], 'BLN')
        set_type(rft[ind], 'RFT')
        set_type(opf[ind], 'OPF')
        set_type(rfr[ind], 'RFR')
        set_type(adc[ind], 'ADC')

    for (time, connection) in connections:

        set_connection(name1=ant, name2=dpf, time=time, connection=connection)

        for ind in (0, 1):

            # Pairs of names to connect
            pairs = [(ant, dpf), (dpf, bln[ind]), (bln[ind], rft[ind]), (rft[ind], opf[ind]), (opf[ind], rfr[ind]), (rfr[ind], adc[ind]), (adc[ind], cor)]

            for pair in pairs:
                set_connection(name1=pair[0], name2=pair[1], time=time, connection=connection)

    print(f"Dish {i} done, took {(datetime.now() - now).total_seconds()}")



# EXPORT THE GRAPH.
# g.io("path_benchmark_small_graph.xml").write().next()




e, took 0.372235
Dish 334 done, took 0.358905
Dish 335 done, took 0.376668
Dish 336 done, took 0.366704
Dish 337 done, took 0.358763
Dish 338 done, took 0.365842
Dish 339 done, took 0.379995
Dish 340 done, took 0.371506
Dish 341 done, took 0.360005
Dish 342 done, took 0.361092
Dish 343 done, took 0.368161
Dish 344 done, took 0.367419
Dish 345 done, took 0.374973
Dish 346 done, took 0.366019
Dish 347 done, took 0.375981
Dish 348 done, took 0.364103
Dish 349 done, took 0.375754
Dish 350 done, took 0.358031
Dish 351 done, took 0.379068
Dish 352 done, took 0.368063
Dish 353 done, took 0.366154
Dish 354 done, took 0.373359
Dish 355 done, took 0.364769
Dish 356 done, took 0.36365
Dish 357 done, took 0.370083
Dish 358 done, took 0.363654
Dish 359 done, took 0.366593
Dish 360 done, took 0.36346
Dish 361 done, took 0.371572
Dish 362 done, took 0.372511
Dish 363 done, took 0.381594
Dish 364 done, took 0.366346
Dish 365 done, took 0.37386
Dish 366 done, took 0.371827
Dish 367 done, took 0.395524


In [23]:
# To benchmark, try finding all of the paths from the correlator to each of the antennas, and time it.
iterations = 1

times_to_query = [2]

total = 0

print(f"Dishes: {dishes}")
print(f"Times to check: {times_to_query}")
print(f"Iterations: {iterations}")

for _ in range(iterations):
    for i in range(len(times_to_query)):
        time = times_to_query[i]
        for dish in range(1, dishes + 1):

            success = False
            while not success:
                try:
                    then = datetime.now()
                    find_paths(name1=cor, name2=f'ANT{str(dish).zfill(6)}', time=time, avoid_type='')

                    delta = (datetime.now() - then).total_seconds()

                    total += delta

                    print(f"Dish {dish} done, time:", delta)

                    success = True
                except GremlinServerError:
                    print("Uh oh! Backend exception!")

        print(f"{len(times_to_query) - i - 1} times left to check!")

print("Total time:", total)
print("Average time per call:", total / (iterations * len(times_to_query) * dishes))
        

.084207
Dish 360 done, time: 1.072152
Dish 361 done, time: 1.092873
Dish 362 done, time: 1.07694
Dish 363 done, time: 1.090899
Dish 364 done, time: 1.066429
Dish 365 done, time: 1.0761
Dish 366 done, time: 1.089332
Dish 367 done, time: 1.155249
Dish 368 done, time: 1.061966
Dish 369 done, time: 1.076739
Dish 370 done, time: 1.079993
Dish 371 done, time: 1.084111
Dish 372 done, time: 1.082448
Dish 373 done, time: 1.059118
Dish 374 done, time: 1.06919
Dish 375 done, time: 1.065146
Dish 376 done, time: 1.083198
Dish 377 done, time: 1.095536
Dish 378 done, time: 1.150839
Dish 379 done, time: 1.0734
Dish 380 done, time: 1.080832
Dish 381 done, time: 1.076589
Dish 382 done, time: 1.082632
Dish 383 done, time: 1.086072
Dish 384 done, time: 1.073893
Dish 385 done, time: 1.068295
Dish 386 done, time: 1.076457
Dish 387 done, time: 1.065543
Dish 388 done, time: 1.081328
Dish 389 done, time: 1.147498
Dish 390 done, time: 1.081038
Dish 391 done, time: 1.073717
Dish 392 done, time: 1.079474
Dish 393